In [1]:
import wandb
from wandb.keras import WandbMetricsLogger


import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D, TimeDistributed, LSTM,Conv1D,MaxPool1D,Bidirectional
from tensorflow.keras.models import Model

from TranscriptionDataset.TensorflowMusicMetaSequence import MusicMetaSequence
from TUtils import get_timestamp

import numpy as np

print("TensorFlow version:", tf.__version__)

2023-06-20 21:15:48.194637: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 21:15:49.352817: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.12.0


In [2]:
logdir = f"TBLogs/{get_timestamp()}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
wandb.tensorboard.patch(root_logdir=logdir)
wandb.init(project='TFArrangement')
print(f"Tensorboard : {logdir}")

wandb: Currently logged in as: draguve. Use `wandb login --relogin` to force relogin


Tensorboard : TBLogs/1687275953


In [3]:
trainSequence = MusicMetaSequence("Trainsets/S_Tier_Mel.hdf5")
testSequence = MusicMetaSequence("Trainsets/Mini_Validation_Test.hdf5",shuffle=False)
# trainSequence[0][0].shape,trainSequence[0][1].shape

In [4]:
input_layer = Input(shape=(None,256),batch_size=1)
x = (Conv1D(data_format="channels_last",filters=128, kernel_size=5, padding="same", activation="relu"))(input_layer)
x = (MaxPool1D(pool_size=2))(x)
x = (Conv1D(data_format="channels_last",filters=64, kernel_size=5, padding="same", activation="relu"))(x)
x = (MaxPool1D(pool_size=2))(x)
x = (Conv1D(data_format="channels_last",filters=8, kernel_size=5, padding="same", activation="relu"))(x)
x = (MaxPool1D(pool_size=2))(x)
x = Bidirectional(LSTM(128))(x)
x = Dense(128,activation="relu")(x)
# output = Dense(1,activation="sigmoid")(x)
outputs = []
for i in range(6):
    outputs.append(Dense(1,name=f'output_{i}',activation="sigmoid")(x))

model = Model(inputs=input_layer, outputs=outputs)

print(model.summary())

2023-06-20 21:16:07.944658: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 21:16:07.951984: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 21:16:07.952048: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 21:16:07.955543: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 21:16:07.955612: I tensorflow/compile

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(1, None, 256)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (1, None, 128)       163968      ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (1, None, 128)       0           ['conv1d[0][0]']                 
                                                                                                  
 conv1d_1 (Conv1D)              (1, None, 64)        41024       ['max_pooling1d[0][0]']          
                                                                                              

2023-06-20 21:16:11.359877: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 21:16:11.361285: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-20 21:16:11.362475: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

__________________________________________________________________________________________________
None


In [5]:
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001)
loss = {}
metrics = {}
for i in range(6):
    loss[f'output_{i}'] = tf.keras.losses.BinaryFocalCrossentropy()
    metrics[f"output_{i}"] = tf.keras.metrics.BinaryAccuracy()
model.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [6]:
model.fit(trainSequence,
          epochs=10,
          verbose=1,
          callbacks=[
              tensorboard_callback,
              WandbMetricsLogger(
                  log_freq="epoch",
                  initial_global_step=np.ceil(model.optimizer.iterations.numpy(),np.ceil(len(trainSequence)))
              )
          ])

Epoch 1/10


2023-06-20 21:16:11.782354: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-20 21:16:12.015311: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 21:16:12.018977: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

1269/1269 [==============================] - 472s 361ms/step - loss: 0.2415 - output_0_loss: 0.0215 - output_1_loss: 0.0999 - output_2_loss: 0.0041 - output_3_loss: 0.0557 - output_4_loss: 0.0564 - output_5_loss: 0.0038 - output_0_binary_accuracy: 0.9898 - output_1_binary_accuracy: 0.8920 - output_2_binary_accuracy: 0.9992 - output_3_binary_accuracy: 0.9519 - output_4_binary_accuracy: 0.9614 - output_5_binary_accuracy: 0.9992
Epoch 2/10
1269/1269 [==============================] - 188s 148ms/step - loss: 0.2196 - output_0_loss: 0.0205 - output_1_loss: 0.0965 - output_2_loss: 0.0028 - output_3_loss: 0.0473 - output_4_loss: 0.0499 - output_5_loss: 0.0028 - output_0_binary_accuracy: 0.9898 - output_1_binary_accuracy: 0.8920 - output_2_binary_accuracy: 0.9992 - output_3_binary_accuracy: 0.9598 - output_4_binary_accuracy: 0.9614 - output_5_binary_accuracy: 0.9992
Epoch 3/10
1269/1269 [==============================] - 182s 143ms/step - loss: 0.2265 - output_0_loss: 0.0216 - output_1_loss: 0

In [ ]:
preds = model.predict(testSequence,verbose=1)
expected_test = testSequence.get_expected_output()

In [ ]:
expected_test

In [ ]:
score = model.evaluate(testSequence, verbose = 1)

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model_name = "ArrangementModel_S_Tier_Only"
model.save(f"Models/{model_name}_{get_timestamp()}.h5")

In [7]:
wandb.finish()

epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▆▅▄▃▄▂▁▂
epoch/output_0_binary_accuracy,▁▁▁▁▁▁▁▁▁▁
epoch/output_0_loss,▅▃▅▅▄▂█▁▃▅
epoch/output_1_binary_accuracy,▁▁▁▁▁▁▁▁▁▁
epoch/output_1_loss,█▅▆▃▃▃▅▂▁▂
epoch/output_2_binary_accuracy,▁▁▁▁▁▁▁▁▁▁
epoch/output_2_loss,█▄▅▇▆▄▅▅▁▆
epoch/output_3_binary_accuracy,▁▃▃▂▃▅▇███
epoch/output_3_loss,█▆▇▇▅▃▃▂▁▂
